In [1]:
# Description: Load tsv file

# Import libraries
import pandas as pd


# Load tsv file
News = pd.read_csv('MINDsmall_train/news.tsv', sep='\t', header=None)
News.columns = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']

User = pd.read_csv('MINDsmall_train/behaviors.tsv', sep='\t', header=None)
User.columns = ['impression_id', 'user_id', 'time', 'history', 'impressions']

topic_size = News['category'].nunique()
subtopic_size = News['subcategory'].nunique()

print(f"Data contains {topic_size} topics and {subtopic_size} subtopics")

Data contains 17 topics and 264 subtopics


In [12]:
from LSTUR import NewsEncoder, TitleEncoder, TopicEncoder,GloVe
import torch as th
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('basic_english')

topic_embedding_dim = 100
subtopic_embedding_dim = 100
word_size = 100


NewsEncoderModule = NewsEncoder(100, 100, topic_size, subtopic_size, 10000)
TitleEncoderModule = TitleEncoder(100)
TopicEncoderModule = TopicEncoder(100, 100, topic_size, subtopic_size)

topic = th.tensor([0],dtype=th.int32)
subtopic = th.tensor([0],dtype=th.int32)

title = tokenizer(News['title'][0])
W = GloVe.get_vecs_by_tokens(title)

In [15]:
NewsEncoderModule(topic, subtopic, W).size()

torch.Size([1, 500])